In [ ]:
# =============================================================================
# Practical 4: Real-time Object Detection with YOLOv8 (YouTube Shorts → full annotated video)
# Course:           Computer Vision
# Objective:        Run YOLOv8 on a YouTube Shorts video and see full video with detections
# Tools:            Python + Ultralytics YOLOv8 + yt-dlp
# =============================================================================


# Step 0: Install required packages (run once)
!pip install ultralytics
!pip install yt-dlp


from ultralytics import YOLO
import cv2
import os
import yt_dlp
from IPython.display import HTML
from base64 import b64encode


print("Ultralytics YOLOv8 ready")


# ========================================================================
# Step 1: Define paths (Colab)
# ========================================================================

# Path to your dataset (if you want to train later)
DATASET_PATH = "/content/images"   # folder containing images/, labels/, data.yaml

# Path to your trained model weights (after training)
MODEL_PATH = os.path.join(
    "runs", "detect", "your_custom_model", "weights", "best.pt"
)

# Directory where the Shorts video will be saved
OUTPUT_DIR = "/content"
INPUT_VIDEO = os.path.join(OUTPUT_DIR, "shorts_video.mp4")
OUTPUT_VIDEO = os.path.join(OUTPUT_DIR, "annotated_shorts.mp4")

# Training run name (used in runs/detect/your_custom_model)
TRAIN_NAME = "your_custom_model"


# ========================================================================
# Step 2: Train custom YOLOv8 model (run once, optional)
# ========================================================================

# Uncomment this block only when you want to train your custom model
"""
print("Starting training...")

model = YOLO("yolov8n.pt")   # nano model (fastest)

model.train(
    data=os.path.join(DATASET_PATH, "data.yaml"),
    epochs=50,      # adjust as needed
    imgsz=640,      # image size
    batch=16,       # reduce if you get out-of-memory errors
    name=TRAIN_NAME # saves to runs/detect/TRAIN_NAME
)

print("Training finished!")
"""


# ========================================================================
# Step 3: Load trained model (or pretrained if not trained yet)
# ========================================================================

# Option A: Use your custom trained model
if os.path.exists(MODEL_PATH):
    print(f"Loading custom model from: {MODEL_PATH}")
    model = YOLO(MODEL_PATH)
else:
    print("Custom model not found, loading pretrained yolov8n.pt")
    model = YOLO("yolov8n.pt")

print("Model loaded ✓")


# ========================================================================
# Step 4: Download YouTube Shorts video (your URL)
# ========================================================================

# Your YouTube Shorts URL
YOUTUBE_URL = "https://www.youtube.com/shorts/Ek5cKd3pnB4?feature=share"

ydl_opts = {
    "format": "bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]/best",
    "outtmpl": INPUT_VIDEO,
    "merge_output_format": "mp4",
}

print("Downloading YouTube Shorts video...")
with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download([YOUTUBE_URL])

print(f"Downloaded video: {INPUT_VIDEO}")


# ========================================================================
# Step 5: Run YOLOv8 and save full annotated video
# ========================================================================

cap = cv2.VideoCapture(INPUT_VIDEO)

if not cap.isOpened():
    print("Error: Cannot open video file")
    exit()

# Get video properties
fps = int(cap.get(cv2.CAP_PROP_FPS))
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define codec and create VideoWriter
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(OUTPUT_VIDEO, fourcc, fps, (width, height))

print("Running YOLOv8 and saving annotated video...")

while True:
    ret, frame = cap.read()
    if not ret:
        print("End of video reached.")
        break

    # Run YOLOv8 inference
    results = model(frame, verbose=False)

    # Draw results on frame (bounding boxes, labels, confidence)
    annotated_frame = results[0].plot()

    # Write annotated frame to output video
    out.write(annotated_frame)

cap.release()
out.release()
print(f"Annotated video saved: {OUTPUT_VIDEO}")


# ========================================================================
# Step 6: Display full annotated video in Colab
# ========================================================================

# Convert video to base64 so it can be embedded in HTML
with open(OUTPUT_VIDEO, "rb") as f:
    video_bytes = f.read()
video_b64 = b64encode(video_bytes).decode()

video_html = f"""
<video width="800" controls>
  <source src="data:video/mp4;base64,{video_b64}" type="video/mp4">
  Your browser does not support the video tag.
</video>
"""

print("Displaying full annotated video below:")
display(HTML(video_html))


# ========================================================================
# Optional: Run on a single image or save output
# ========================================================================

# Example: run on one image
results = model("/content/438159029_1112336846689554_6666657298374937680_n_jpg.rf.ef8d85be9ab7c620ead05befd5352e50.jpg")
results[0].save("result.jpg")

print("You can uncomment the optional section above to run on a single image.")


Ultralytics YOLOv8 ready
Custom model not found, loading pretrained yolov8n.pt
Model loaded ✓
[youtube] Extracting URL: https://www.youtube.com/shorts/Ek5cKd3pnB4?feature=share
[youtube] Ek5cKd3pnB4: Downloading webpage


[youtube] Ek5cKd3pnB4: Downloading android vr player API JSON
[youtube] Ek5cKd3pnB4: Downloading ios downgraded player API JSON
[youtube] Ek5cKd3pnB4: Downloading m3u8 information
[info] Testing format 620
[info] Ek5cKd3pnB4: Downloading 1 format(s): 620+140
[download] /content/shorts_video.mp4 has already been downloaded
Downloaded video: /content/shorts_video.mp4
Running YOLOv8 and saving annotated video...
End of video reached.
Annotated video saved: /content/annotated_shorts.mp4
Displaying full annotated video below:
Buffered data was truncated after reaching the output size limit.